In [13]:
import json
import requests
from bs4 import BeautifulSoup
import time
import random

In [14]:
main_url = "https://m.my.kt.com/product/s_OttSubscribeView.do"
    
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://m.my.kt.com/"
    }
print(f"메인 페이지 접속({main_url})")

메인 페이지 접속(https://m.my.kt.com/product/s_OttSubscribeView.do)


In [15]:
response = requests.get(main_url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
        
products = []
        
product_list = soup.select("ul.opage-subscription li[data-wdiclinkmobileurl]")
        
print(f" 총 {len(product_list)}개의 구독 상품")

 총 0개의 구독 상품


In [1]:
%pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random

# 1. Selenium 브라우저 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 화면에 창을 띄우지 않고 백그라운드에서 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# 차단 방지를 위한 User-Agent 설정
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# 2. 브라우저 실행
print("가상 브라우저를 실행")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # 3. 메인 페이지 접속
    main_url = "https://m.my.kt.com/product/s_OttSubscribeView.do"
    print(f"페이지 접속 ({main_url})")
    driver.get(main_url)

    #자바스크립트가 목록을 불러올 때까지 기다리기
    print("데이터 로딩 대기")
    time.sleep(5)

    # 4. 로딩된 페이지의 HTML 가져오기 (이제 데이터가 들어있음!)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 5. 목록 찾기 KT 페이지 구조상 data-wdiclinkmobileurl 속성이 있는 리스트를 찾기
    product_list = soup.select("ul.opage-subscription li[data-wdiclinkmobileurl]")

    print(f"총 {len(product_list)}개의 구독 상품을 발견했습니다!")

    # 6. 결과 출력 및 데이터 추출 테스트
    products = []
    for li in product_list:
        name = li.get('data-ottprodnm')
        detail_url = li.get('data-wdiclinkmobileurl')
        print(f"   - 상품명: {name}")
        # print(f"   - 링크: {detail_url}") 
        
        products.append({
            "name": name,
            "detail_url": detail_url
        })

except Exception as e:
    print(f" 에러 발생: {e}")

finally:
    # 브라우저 종료 (메모리 확보)
    driver.quit()
    print(" 브라우저 종료 완료")

가상 브라우저를 실행
페이지 접속 (https://m.my.kt.com/product/s_OttSubscribeView.do)


In [ ]:
import json
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def crawl_kt_hybrid():
    # 1. Selenium 설정 (목록 페이지용)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    results = []

    try:
        # 2. 메인 목록 페이지 접속 (Selenium)
        main_url = "https://m.my.kt.com/product/s_OttSubscribeView.do"
        print(f"메인 페이지 접속({main_url})")
        driver.get(main_url)
        time.sleep(5) # 데이터 로딩 대기

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_list = soup.select("ul.opage-subscription li[data-wdiclinkmobileurl]")
        
        print(f"총 {len(product_list)}개의 상품")

        # 3. 각 상품별 정보 추출 및 상세 크롤링
        for i, li in enumerate(product_list):
            item = {}
            
            # [A] 기본 정보 (Graph DB용)
            item['name'] = li.get('data-ottprodnm')
            item['detail_url'] = li.get('data-wdiclinkmobileurl')
            
            # 가격 추출 (할인가 우선)
            sale_tag = li.select_one(".discount-price")
            orig_tag = li.select_one(".price")
            
            # aria-label 값 가져오기 (예: "월 13,900원")
            item['price_sale'] = sale_tag['aria-label'] if sale_tag else "정보없음"
            item['price_original'] = orig_tag['aria-label'] if orig_tag else "정보없음"
            
            # [B] 상세 약관 크롤링 (Vector DB용) - 속도를 위해 Requests 사용
            if item['detail_url']:
                try:
                    # 상세 페이지는 보통 정적 페이지라 Requests가 빠름
                    headers = {"User-Agent": "Mozilla/5.0"}
                    resp = requests.get(item['detail_url'], headers=headers)
                    sub_soup = BeautifulSoup(resp.text, 'html.parser')
                    
                    target_div = sub_soup.find("div", class_="pduct-ott")
                    if target_div:
                        # <br> -> 공백 변환
                        for br in target_div.find_all("br"): br.replace_with(" ")
                        # 텍스트 추출
                        item['terms_text'] = target_div.get_text(separator="\n", strip=True)
                    else:
                        item['terms_text'] = "상세 약관 없음"
                        
                except Exception as e:
                    print(f"   ❌ 상세 수집 실패 ({item['name']}): {e}")
                    item['terms_text'] = ""
            
            print(f"   [{i+1}] {item['name']} 완료 (가격: {item['price_sale']})")
            results.append(item)
            
    except Exception as e:
        print(f"❌ 크롤링 중 치명적 오류: {e}")
        
    finally:
        driver.quit()
        
    return results

# === 실행 및 저장 ===
data = crawl_kt_hybrid()

# JSON 저장
with open("kt_products_hybrid.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("\n✅ [Step 1] 데이터 수집 및 JSON 저장 완료!")

🚀 메인 페이지 접속 및 로딩 대기... (https://m.my.kt.com/product/s_OttSubscribeView.do)
📦 총 33개의 상품 발견! 상세 수집 시작...
   [1] 디즈니플러스 프리미엄+메가MGC커피 완료 (가격: 월 13,900원)
   [2] 디즈니플러스 스탠다드+메가MGC커피 완료 (가격: 월 9,900원)
   [3] 디즈니+ 프리미엄 생활구독팩 완료 (가격: 월 13,900원)
   [4] 디즈니+ 스탠다드 생활구독팩 완료 (가격: 월 9,900원)
   [5] 티빙 프리미엄+메가MGC커피 완료 (가격: 월 17,000원)
   [6]  티빙 스탠다드+메가MGC커피 완료 (가격: 월 13,500원)
   [7] 티빙 베이직+메가MGC커피 완료 (가격: 월 9,500원)
   [8] 티빙 광고형 스탠다드+메가MGC커피 완료 (가격: 월 5,500원)
   [9] 딥엘 인디비주얼 완료 (가격: 월 9,900원)
   [10] 유튜브 프리미엄 생활구독팩 완료 (가격: 월 14,900원)
   [11] 모아진(국내해외매거진무제한) 완료 (가격: 월 13,000원)
   [12] 모아진(국내매거진무제한) 완료 (가격: 월 7,000원)
   [13] 모아진(매거진5종) 완료 (가격: 월 4,000원)
   [14] 콴다 프리미엄 완료 (가격: 월 16,500원)
   [15] 콴다 프리미엄 Lite 완료 (가격: 월 9,000원)
   [16] 유튜브 프리미엄+롯데시네마 완료 (가격: 월 27,900원)
   [17] 티빙 프리미엄+스타벅스 완료 (가격: 월 17,000원)
   [18] 티빙 스탠다드+스타벅스 완료 (가격: 월 13,500원)
   [19] 티빙 베이직+스타벅스 완료 (가격: 월 9,500원)
   [20] 유튜브 프리미엄+스타벅스 완료 (가격: 월 14,900원)
   [21] 디즈니플러스 프리미엄+스타벅스 완료 (가격: 월 13,900원)
   [22] 디즈니플러스 스탠다드+스타벅스 완료 (가격: 월 9,9

In [5]:
import json
import networkx as nx
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

print("⚙️ 데이터베이스 구축 시작...")

# 1. JSON 로드
with open("kt_products_hybrid.json", "r", encoding="utf-8") as f:
    products_data = json.load(f)

# === [A] Graph DB 구축 (상품 -[COSTS]-> 가격) ===
G = nx.DiGraph()

for prod in products_data:
    p_name = prod['name']
    p_price = prod['price_sale']
    
    # 노드 추가
    G.add_node(p_name, type="product")
    G.add_node(p_price, type="price")
    
    # 엣지(관계) 추가
    G.add_edge(p_name, p_price, relation="COSTS")
    
    # (심화) 할인가격과 원가 관계도 추가 가능
    # if prod['price_original'] != "정보없음":
    #     G.add_edge(p_name, prod['price_original'], relation="ORIGINAL_PRICE")

print(f"✅ Graph DB 구축 완료 (노드 {G.number_of_nodes()}개)")


# === [B] Vector DB 구축 (상세 약관) ===
text_docs = []
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

for prod in products_data:
    if prod['terms_text']:
        # 약관 텍스트를 청크로 자름
        chunks = splitter.create_documents(
            texts=[prod['terms_text']], 
            metadatas=[{"product_name": prod['name']}] # 메타데이터에 상품명 심기
        )
        text_docs.extend(chunks)

# 임베딩 및 저장
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(text_docs, embeddings)

print(f"✅ Vector DB 구축 완료 (문서 청크 {len(text_docs)}개)")

⚙️ 데이터베이스 구축 시작...
✅ Graph DB 구축 완료 (노드 44개)


C:\Users\lg\AppData\Local\Temp\ipykernel_20848\754429375.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Vector DB 구축 완료 (문서 청크 204개)


In [6]:
def ask_hybrid_bot(query):
    print(f"\n💬 질문: {query}")
    print("Thinking...", end="", flush=True)
    
    context_result = []
    
    # 1. [Graph Search] 가격, 얼마, 비용 관련 질문
    price_keywords = ["얼마", "가격", "비용", "요금", "싸게"]
    if any(k in query for k in price_keywords):
        # 그래프에서 상품명과 가격 관계를 탐색
        found_in_graph = False
        for node in G.nodes():
            # 질문에 상품명이 포함되어 있다면
            if isinstance(node, str) and node in query:
                # 그 상품의 이웃 노드(가격)를 찾음
                neighbors = G.neighbors(node)
                for n in neighbors:
                    if G.get_edge_data(node, n)['relation'] == 'COSTS':
                        context_result.append(f"[가격정보] '{node}'의 구독료는 {n} 입니다.")
                        found_in_graph = True
        
        if not found_in_graph:
             # 상품명을 못 찾았으면 전체 가격 리스트를 참고용으로 줌
             pass 

    # 2. [Vector Search] 약관, 해지, 유의사항 관련 질문
    # (항상 실행하거나, 특정 키워드에만 실행할 수 있음)
    # 질문과 유사한 약관 내용 검색
    vector_results = vector_db.similarity_search(query, k=2)
    for doc in vector_results:
        context_result.append(f"[약관-'{doc.metadata['product_name']}'] {doc.page_content[:200]}...")

    # 3. 답변 생성 (LLM에게 정보 전달)
    final_context = "\n".join(context_result)
    
    if not final_context:
        print("\r🤖 답변: 죄송합니다. 관련 정보를 찾을 수 없습니다.")
        return

    # (여기서 LLM 호출 - 이전 코드 활용)
    # 간단한 출력을 위해 print로 대체하지만, 실제로는 rag_chain.invoke()에 final_context를 넣으면 됩니다.
    print(f"\r🤖 [AI가 참고한 정보]\n{final_context}")
    print("-" * 50)
    print("💡 (이 정보를 바탕으로 LLM이 자연스럽게 답변하게 됩니다)")

# === 테스트 ===
ask_hybrid_bot("디즈니플러스 프리미엄+메가MGC커피 가격이 얼마야?")
ask_hybrid_bot("디즈니플러스 해지하면 위약금 있어?")


💬 질문: 디즈니플러스 프리미엄+메가MGC커피 가격이 얼마야?
🤖 [AI가 참고한 정보]
[가격정보] '디즈니플러스 프리미엄+메가MGC커피'의 구독료는 월 13,900원 입니다.
[약관-'티빙 프리미엄+메가MGC커피'] 아메리카노(HOT) 상품이 제공되며, 제공된 기프티쇼는 유효기간 내 이용 가능합니다. (유효기간 30일)
부가서비스 결제/취소/환불 관련해서는 아래의 정책이 적용됩니다.
1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이내 해지하시는 경우에는, 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 여부에 따라 해지 처리됩니다....
[약관-'티빙 광고형 스탠다드+메가MGC커피'] 아메리카노(HOT) 상품이 제공되며, 제공된 기프티쇼는 유효기간 내 이용 가능합니다. (유효기간 30일)
부가서비스 결제/취소/환불 관련해서는 아래의 정책이 적용됩니다.
1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이내 해지하시는 경우에는, 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 여부에 따라 해지 처리됩니다....
--------------------------------------------------
💡 (이 정보를 바탕으로 LLM이 자연스럽게 답변하게 됩니다)

💬 질문: 디즈니플러스 해지하면 위약금 있어?
🤖 [AI가 참고한 정보]
[약관-'디즈니+ 프리미엄 생활구독팩'] 이미 디즈니+를 직접 결제 중인 경우, 이용 중인 디즈니+ 이메일 계정을 등록해야 이중 과금 되지 않습니다. 단, 인앱 결제(구글 및 애플)로 이용 중인 경우 인앱 결제 사업자의 고객센터를 통해 조치(해지, 환불신청 등) 하셔야 합니다....
[약관-'디즈니+ 스탠다드 생활구독팩'] 이미 디즈니+를 직접 결제 중인 경우, 이용 중인 디즈니+ 이메일 계정을 등록해야 이중 과금 되지 않습니다. 단, 인앱 결제(구글 및 애플)로 이용 중인 경우 인앱 결제 사업자의 고객센터를 통해 조치(해지, 환불신청 등) 하셔야 합니다

In [7]:
ask_hybrid_bot("디즈니 플러스가 포함된 거 가입하고 싶으면 어떤 게 있어?")


💬 질문: 디즈니 플러스가 포함된 거 가입하고 싶으면 어떤 게 있어?
🤖 [AI가 참고한 정보]
[약관-'디즈니+ 프리미엄 생활구독팩'] 이미 디즈니+를 직접 결제 중인 경우, 이용 중인 디즈니+ 이메일 계정을 등록해야 이중 과금 되지 않습니다. 단, 인앱 결제(구글 및 애플)로 이용 중인 경우 인앱 결제 사업자의 고객센터를 통해 조치(해지, 환불신청 등) 하셔야 합니다....
[약관-'디즈니+ 스탠다드 생활구독팩'] 이미 디즈니+를 직접 결제 중인 경우, 이용 중인 디즈니+ 이메일 계정을 등록해야 이중 과금 되지 않습니다. 단, 인앱 결제(구글 및 애플)로 이용 중인 경우 인앱 결제 사업자의 고객센터를 통해 조치(해지, 환불신청 등) 하셔야 합니다....
--------------------------------------------------
💡 (이 정보를 바탕으로 LLM이 자연스럽게 답변하게 됩니다)


In [12]:
ask_hybrid_bot("콴다 포함된 요금제 전부 알려줘")


💬 질문: 콴다 포함된 요금제 전부 알려줘
🤖 [AI가 참고한 정보]
[약관-'디즈니플러스 프리미엄+스타벅스'] 비밀번호를 입력한 뒤 [다음]을 누르세요.
※ 디즈니+ App이 없으면, 2단계 이후 나오는 화면으로 플레이스토어/앱스토어에서 다운로드 하세요.
계정 정상 생성되었는지 확인 필요시
디즈니App-계정에서 '디즈니+ KT(케이티) 계정으로 이동해 디즈니+ 멤버십을 관리하세요'가 표기되면 정상 생성된 것 입니다.
※ '>' 클릭 시 'kt.com-마이'페이지로 ...
[약관-'디즈니플러스 스탠다드+스타벅스'] 비밀번호를 입력한 뒤 [다음]을 누르세요.
※ 디즈니+ App이 없으면, 2단계 이후 나오는 화면으로 플레이스토어/앱스토어에서 다운로드 하세요.
계정 정상 생성되었는지 확인 필요시
디즈니App-계정에서 '디즈니+ KT(케이티) 계정으로 이동해 디즈니+ 멤버십을 관리하세요'가 표기되면 정상 생성된 것 입니다.
※ '>' 클릭 시 'kt.com-마이'페이지로 ...
--------------------------------------------------
💡 (이 정보를 바탕으로 LLM이 자연스럽게 답변하게 됩니다)
